# Dimensionality Reduction
* Application of the persistent extension method to dimensionality reduction.
* X: points sampled from a cylinder
* Y: projection of X to 2-dimensions via PCA
* This notebook overlaps with "examples/EXAMPLE_EXTENSION_VR_VR.ipynb"


In [2]:
using Revise
includet("../../../extension_method.jl")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-4654703612885072738\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-13777626718854618406\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-4654703612885072738\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

┌ Warning: ORCA.jl has been deprecated and all savefig functionality
│ has been implemented directly in PlotlyBase itself.
│ 
│ By implementing in PlotlyBase.jl, the savefig routines are automatically
│ available to PlotlyJS.jl also.
└ @ ORCA /opt/julia/packages/ORCA/U5XaN/src/ORCA.jl:8


In [3]:
using .ext
using Printf
using Combinatorics
using Distances
using Distributions
using Eirene
using Plots
using JLD
using LinearAlgebra
using Measures
using MultivariateStats
using StatsBase

# 1. Load points
* points: cylinder with disc as a base

In [4]:
# load points
data = load("points.jld")
points = data["points"];

┌ Warning: JLD incorrectly extends FileIO functions (see FileIO documentation)
└ @ FileIO /opt/julia/packages/FileIO/JA3Vl/src/loadsave.jl:215


The points were generated using the following code 

In [ ]:
# generate a cylinder in 3D 
"""
n = 200
theta = rand(Uniform(0,2*pi), n)
height = rand(Uniform(0, 4), n)

points = zeros(n, 3)
for i=1:n
    points[i, 1] = 2 * cos(theta[i]) 
    points[i, 2] = 2 * sin(theta[i])
    points[i, 3] = height[i]
end
"""

# save points
#save("points.jld", "points", points)

Plot points on cylinder

In [5]:
# plot cylinder and sampled points

# equation of the cylinder
X(theta,z) = 4 * cos(theta)
Y(theta,z) = 4 * sin(theta)
Z(theta,z) = z

ts = range(-0.001, 2pi, length=50)
zs = range(-0.001, 2, length = 50)

surface(X.(ts',zs), Y.(ts', zs), Z.(ts', zs), aspect_ratio = :equal, legend = :none, alpha = 0.8)

# show sampled points
plot!(points[:,1], points[:,2], points[:,3], 
    seriestype = :scatter, 
    label = "",
    xticks = nothing,
    yticks = nothing,
    zticks = nothing,
    framestyle = :box,
    markercolor = :teal,
    markersize = 3,
    xlim = (-4,4),
    ylim = (-4, 4),
    zlim = (0, 2),
    aspect_ratio = :equal)

<!DOCTYPE html>
 
 
 Plots.jl

# 2. Apply PCA
* Note: regardless of the radius of the disc and the height of the cylidner, PCA will always destroy the S1 structure.
    * Why? Because PCA usually requires feature scaling. After feature scaling, PCA will destroy the S1 structure.

### 2(a) scale features

In [6]:
# scale features
Xp = transpose(points)

# mean normalization & feature scaling 
dt = fit(ZScoreTransform, Xp, dims = 2)
X_t = StatsBase.transform(dt, Xp);

print("means \n")
print(mean(X_t[1,:]), "\n")
print(mean(X_t[2,:]), "\n")
print(mean(X_t[3,:]), "\n")
print("STD \n")
print(std(X_t[1,:]), "\n")
print(std(X_t[2,:]), "\n")
print(std(X_t[3,:]), "\n")

means 
2.4424906541753444e-17
2.3462135012586317e-17
-9.159339953157542e-18
STD 
1.0
1.0
1.0


## 2(b) perform PCA

In [7]:
# train PCA
M = fit(PCA, X_t; maxoutdim = 2)

# apply PCA
X_pca = transform(M, X_t);

Plot results of PCA in 2D

In [8]:
# plot PCA
plot(X_pca[1,:], X_pca[2,:], 
    seriestype = :scatter, 
    label = "",
    framestyle = :box,
    xaxis = nothing,
    yaxis = nothing,
    markersize = 8
    #title = "Visualization of PCA"
    )

<!DOCTYPE html>
 
 
 Plots.jl

Plot in 3D

In [9]:
# find equation of the plane ax + by + cz = D

# find basis of the 2D subspace 
basis = projection(M)

# cross product
basis_x = cross(basis[:,1], basis[:,2])

# find D
D = dot(basis[:,1], basis_x)

# equation of the plane
f(x,y) =  (D - basis_x[1] * x - basis_x[2] * y) / basis_x[3] ;

In [10]:
# plot the (standardized) points
plot(X_t[1,:], X_t[2,:], X_t[3,:], seriestype = :scatter, label = "", legend =:none,
    framestyle = :box,
    markercolor = :teal,
    markersize = 2,
    xlim = (-1.5, 1.5),
    ylim = (-1.5, 1.5),
    zlim = (-2, 2),
    xaxis = nothing,
    yaxis = nothing,
    zaxis = nothing,
    aspect_ratio = :equal)

# plot cylinder?
# equation of cylinder with y as the dependent variable
X(theta,z) = 1.5 * cos(theta)
Y(theta,z) = 1.5 * sin(theta)
Z(theta,z) = z

ts = range(-0.001, 2pi, length=50)
zs = range(-2, 2, length = 50)

surface!(X.(ts',zs), Y.(ts', zs), Z.(ts', zs), aspect_ratio = :equal, legend = :none, color=:greys, alpha = 0.6)

# plot the 2D subspace
x = range(-1.5, stop = 1.5, length = 100)
y = range(-1.5, stop = 1.5, length = 100)
plot!(x, y, f, st = :surface)


<!DOCTYPE html>
 
 
 Plots.jl

# 3. Run extension method

Prepare distance matrices

In [11]:
# original distance
D = Distances.pairwise(Euclidean(), points, points, dims=1)

# distance in reduced space
D_pca = Distances.pairwise(Euclidean(), X_pca,X_pca)

D_Y = D_pca
D_Z = D;


## 3(a) Plot relevant barcodes

In [12]:
# run Eirene
C_Z = eirene(D_Z, record = "all")
C_Y = eirene(D_Y, record = "all");

In [13]:
# plot barcodes
barcode_Z = barcode(C_Z, dim = 1)
barcode_Y = barcode(C_Y, dim = 1)

l = grid(2, 1)
p1 = plot_barcode(barcode_Z, title = "barcode of original points", lw = 3)
p2 = plot_barcode(barcode_Y, title = "barcode of PCA", lw = 3)
plot(p1, p2, layout = l, size = (500, 500))

<!DOCTYPE html>
 
 
 Plots.jl

## 3(b) Run extension method

In [14]:
# select bar of interest
Z_bar = 34

# run extension
extension_pca = run_extension_VR_to_VR_bar(C_Z = C_Z, 
                                        D_Z = D_Z, 
                                        C_Y = C_Y, 
                                        D_Y = D_Y, 
                                        Z_bar = Z_bar, 
                                        dim = 1);

## 3(c) Explore the cycle extension & bar extension under fixed interval decomposition of `C_Y`

Plot all values of `extension["nontrivial_pY"]` on the target barcode $\text{BC}_k(Y^{\bullet})$.

In [15]:
plot_pY(extension_pca, lw = 4)

<!DOCTYPE html>
 
 
 Plots.jl

Find all cycle and bar extensions

In [16]:
CE, BE = find_CE_BE(extension_pca);

Plot a cycle extension

In [17]:
# select parameter
param = extension_pca["nontrivial_pY"][1]
@printf("number of cycle extensions at parameter %.4f : %i", param, length(CE[param]))

number of cycle extensions at parameter 0.4675 : 8

In [18]:
# plot the 8 cycle extensions at selected parameter
ms = 3
titlefontsize = 16

p_objects = []
for i=0:7
    p = plot_cycle_single(X_pca, cycle = CE[param][i], markersize = ms, title = "cycle extension "*string(i), titlefontsize = 10)
    push!(p_objects,p)
end

plot(p_objects..., layout = grid(2, 4), size = (700, 300))

<!DOCTYPE html>
 
 
 Plots.jl

Plot a bar extension

In [19]:
# select parameter 
param = extension_pca["nontrivial_pY"][1]
@printf("number of cycle extensions at parameter %.4f : %i", param, length(CE[param]))

number of cycle extensions at parameter 0.4675 : 8

In [21]:
# select cycle extension 
y= 0

# plot the corresponding bar extension
be = BE[param][y]

# plot the bar extension
p = plot_barcode(barcode_Y, title = "selected bar extension", lw = 3, selected_bars = be, epsilon= param, v_line = [param])
plot(p, size = (500, 300))

<!DOCTYPE html>
 
 
 Plots.jl

## 3(d) Explore alternative bar extensions

In [22]:
# find all bar extensions at all parameters (for a FIXED interval decomposition of C_Y)
_, BE = find_CE_BE(extension_pca)

# find all alternative bar extensions (under alternative interval decompositions of C_Y)
BE_alt = find_alt_BE(extension_pca, BE);

In [23]:
# select parameter
param = extension_pca["nontrivial_pY"][1]

BE_alt[param]

8-element Array{Any,1}:
 [27, 36]
 [26, 27, 36]
 [27, 34, 36]
 [27]
 [26, 27, 34, 36]
 [26, 27]
 [27, 34]
 [26, 27, 34]

In [24]:
# plot one of the alternative bar extensions

# select an alternative bar extension
alt = BE_alt[param][2]

p =plot_barcode(barcode_Y, selected_bars = alt, lw = 3,
                    epsilon = param, v_line = [param],
                    title = "alternative bar extensions")
plot(p)

<!DOCTYPE html>
 
 
 Plots.jl